## Caso de Negocio

A una empresa vendedora de bebidas le interesa conocer las ventas que más frecuentan los consumidores al momento de comprar un refresco, por ende nos fijamos en los pedidos y categorizamos algunas columnas con el fin de poder asociarlas y obtener insights

Algunas columnas categorizadas son: 
* `Temperatura` -> `Clima` 
* `Fecha` -> `Estación`
* `Cantidad` -> `Categoría_Cantidad`

### Preparación de Ambiente

In [416]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import defaultdict
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules
from mlxtend.frequent_patterns import fpgrowth

### Cargar Dataframe

In [417]:
df = pd.read_csv('data/ventas_bebidas.csv', encoding='latin-1')
df.columns = df.columns.str.lower()

### Limpieza y Transformación de Datos

In [418]:
df['fecha'] = pd.to_datetime(df['fecha'], format='%d-%m-%y')

def obtener_estacion(fecha):
    mes = fecha.month
    dia = fecha.day

    # Invierno: 21 diciembre a 20 marzo
    if (mes == 12 and dia >= 21) or mes in [1, 2] or (mes == 3 and dia <= 20):
        return 'Invierno'
    # Primavera: 21 marzo a 20 junio
    elif (mes == 3 and dia >= 21) or mes in [4, 5] or (mes == 6 and dia <= 20):
        return 'Primavera'
    # Verano: 21 junio a 22 septiembre
    elif (mes == 6 and dia >= 21) or mes in [7, 8] or (mes == 9 and dia <= 22):
        return 'Verano'
    # Otoño: 23 septiembre a 20 diciembre
    elif (mes == 9 and dia >= 23) or mes in [10, 11] or (mes == 12 and dia <= 20):
        return 'Otoño'

def categorizar_temperatura(temp):
    if temp < 15:
        return 'Frío'
    elif 15 <= temp < 25:
        return 'Templado'
    elif 25 <= temp < 35:
        return 'Cálido'
    else:
        return 'Muy Cálido'  

def categorizar_unidades(unidades):
    if unidades <= 50:
        return 'poco'
    elif 51 <= unidades <= 200:
        return 'moderado'
    else:
        return 'mucho'

In [419]:
columns_to_drop = ['producto', 'producto_desc', 'monto' , 'precio_unidad', 'contador', 'segmento']
df.drop(columns=columns_to_drop,inplace=True)

In [420]:
df['estacion'] = df['fecha'].apply(obtener_estacion)
df.drop(columns=['fecha'], inplace=True)

df['clima'] = df['prom_temperatura'].apply(categorizar_temperatura)
df.drop(columns=['prom_temperatura'], inplace=True)

df['cat_unidades'] = df['unidades'].apply(categorizar_unidades)
df.drop(columns=['unidades'], inplace=True)

In [421]:
tienda_cat = defaultdict()
for i, tienda in enumerate(df['tienda'].unique(), start=1):
    tienda_cat[tienda] = f'Tienda {i}'

df['tienda'] = df['tienda'].replace(tienda_cat)

### Selección de Variables

In [422]:
df['marca_municipio'] = df['marca'] +' - '+ df['municipio']
df['marca_estacion'] = df['marca'] +' - '+ df['estacion']
df['marca_clima'] = df['marca'] + ' - ' + df['clima']
df['marca_cat'] = df['marca'] + ' - ' + df['cat_unidades']

df.drop(columns=['estacion','clima','cat_unidades'],inplace=True)

## Aprori


### Situación 1: Análisis en los pedidos a través de marca + estación

In [423]:
basket = df.pivot_table(index=['pedido'], columns='marca_estacion', aggfunc='size', fill_value=0)
basket = (basket > 0).astype(bool)

frequent_itemsets = apriori(basket, min_support=0.2, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

In [424]:
def obtain_3_top_rules(rules, wordkeys):
    rules = rules.sort_values(by=['confidence','lift'],ascending=False)

    for i, item in enumerate(rules['antecedents'].values):
        for row in item:
            for clave in wordkeys:
                if clave in row:
                    rules.loc[i,'temp'] = clave

    result = pd.DataFrame(columns=rules.columns)
    groups = rules.groupby('temp')

    for name in groups.groups.keys():
        group = groups.get_group(name).head(3)  # Obtener los primeros 2
        result = pd.concat([result, group])      # Concatenar al resultado

    result.reset_index(drop=True, inplace=True)

    return(result.sort_values(by=['confidence','lift'],ascending=False).drop(columns='temp').reset_index(drop=True))

In [425]:
obtain_3_top_rules(rules, ['Primavera', 'Invierno', 'Otoño', 'Verano'])

C:\Users\Usuario\AppData\Local\Temp\ipykernel_4200\3237406060.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, group])      # Concatenar al resultado


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(COCA-COLA LIGHT - Primavera, JOYA - Primavera)",(COCA-COLA - Primavera),0.201033,0.275725,0.200636,0.998024,3.619634,0.145206,366.483115,0.905831
1,(JOYA - Primavera),(COCA-COLA - Primavera),0.239968,0.275725,0.235598,0.981788,3.560750,0.169433,39.769278,0.946224
2,(JOYA - Invierno),(COCA-COLA - Invierno),0.223679,0.263806,0.219309,0.980462,3.716600,0.160301,37.679741,0.941539
3,(COCA-COLA LIGHT - Primavera),(COCA-COLA - Primavera),0.234803,0.275725,0.229241,0.976311,3.540887,0.164500,30.574749,0.937778
4,(COCA-COLA LIGHT - Invierno),(COCA-COLA - Invierno),0.216528,0.263806,0.209774,0.968807,3.672422,0.152652,23.601510,0.928814
5,(COCA-COLA LIGHT - Primavera),"(JOYA - Primavera, COCA-COLA - Primavera)",0.234803,0.235598,0.200636,0.854484,3.626874,0.145316,5.253042,0.946528


### Situación 2: Análisis en los pedidos a través de marca + clima

In [426]:
basket = df.pivot_table(index=['pedido'], columns='marca_clima', aggfunc='size', fill_value=0)
basket = (basket > 0).astype(bool)


frequent_itemsets = apriori(basket, min_support=0.45, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

In [427]:
obtain_3_top_rules(rules, ['Muy Cálido', 'Cálido', 'Templado', 'Frío'])

C:\Users\Usuario\AppData\Local\Temp\ipykernel_4200\3237406060.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, group])      # Concatenar al resultado


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(JOYA - Cálido),(COCA-COLA - Cálido),0.490663,0.546683,0.481128,0.980567,1.793668,0.212891,23.326960,0.868744
1,(COCA-COLA LIGHT - Cálido),(COCA-COLA - Cálido),0.477155,0.546683,0.463647,0.971690,1.777431,0.202795,16.012772,0.836558
2,(COCA-COLA - Cálido),(JOYA - Cálido),0.546683,0.490663,0.481128,0.880087,1.793668,0.212891,4.247558,0.976100


### Situación 3: Análisis en los pedidos a través de marca + categoría

In [428]:
basket = df.pivot_table(index=['pedido'], columns='marca_cat', aggfunc='size', fill_value=0)
basket = (basket > 0).astype(bool)

frequent_itemsets = apriori(basket, min_support=0.60, use_colnames=True)
rules = association_rules(frequent_itemsets, metric="confidence", min_threshold=0.5)

In [429]:
obtain_3_top_rules(rules, ['poco', 'moderado', 'mucho'])

C:\Users\Usuario\AppData\Local\Temp\ipykernel_4200\3237406060.py:15: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  result = pd.concat([result, group])      # Concatenar al resultado


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,"(SPRITE - poco, FANTA - poco)",(JOYA - poco),0.613826,0.811681,0.613429,0.999353,1.231214,0.115198,290.953516,0.486293
1,"(FANTA - poco, COCA-COLA - poco)",(JOYA - poco),0.613826,0.811681,0.611045,0.995469,1.226430,0.112814,41.564788,0.478088
2,(FANTA - poco),(JOYA - poco),0.627731,0.811681,0.624553,0.994937,1.225774,0.115036,37.193087,0.494774


# FP-Growth

## Situación 1

In [430]:
# Crear la tabla de transacciones binaria
basket2 = df.pivot_table(index=['pedido'], columns='marca_estacion', aggfunc='size', fill_value=0)
basket2 = (basket2 > 0).astype(int)

# Aplicar el algoritmo FP-Growth
frequent_itemsets2 = fpgrowth(basket2, min_support=0.21, use_colnames=True)

g:\Supermarket-Sales-Tracker-Mx\.venv\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [431]:
rules2 = association_rules(frequent_itemsets2, metric="confidence", min_threshold=0.5)

In [432]:
rules2[:]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(JOYA - Invierno),(COCA-COLA - Invierno),0.223679,0.263806,0.219309,0.980462,3.716600,0.160301,37.679741,0.941539
1,(COCA-COLA - Invierno),(JOYA - Invierno),0.263806,0.223679,0.219309,0.831325,3.716600,0.160301,4.602475,0.992859
2,(JOYA - Primavera),(COCA-COLA - Primavera),0.239968,0.275725,0.235598,0.981788,3.560750,0.169433,39.769278,0.946224
3,(COCA-COLA - Primavera),(JOYA - Primavera),0.275725,0.239968,0.235598,0.854467,3.560750,0.169433,5.222397,0.992938
4,(COCA-COLA LIGHT - Primavera),(COCA-COLA - Primavera),0.234803,0.275725,0.229241,0.976311,3.540887,0.164500,30.574749,0.937778
5,(COCA-COLA - Primavera),(COCA-COLA LIGHT - Primavera),0.275725,0.234803,0.229241,0.831412,3.540887,0.164500,4.538859,0.990763


## Situación 2

In [433]:
basket2 = df.pivot_table(index=['pedido'], columns='marca_clima', aggfunc='size', fill_value=0)
basket2 = (basket2 > 0).astype(int)

# Aplicar el algoritmo
frequent_itemsets2 = fpgrowth(basket2, min_support=0.43, use_colnames=True)

g:\Supermarket-Sales-Tracker-Mx\.venv\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [434]:
frequent_itemsets2

,support,itemsets
0,0.546683,(COCA-COLA - Cálido)
1,0.490663,(JOYA - Cálido)
2,0.477155,(COCA-COLA LIGHT - Cálido)
3,0.481128,"(JOYA - Cálido, COCA-COLA - Cálido)"
4,0.463647,"(COCA-COLA LIGHT - Cálido, COCA-COLA - Cálido)"


In [435]:
rules2 = association_rules(frequent_itemsets2, metric="confidence", min_threshold=0.5)

In [436]:
rules2[:]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(JOYA - Cálido),(COCA-COLA - Cálido),0.490663,0.546683,0.481128,0.980567,1.793668,0.212891,23.326960,0.868744
1,(COCA-COLA - Cálido),(JOYA - Cálido),0.546683,0.490663,0.481128,0.880087,1.793668,0.212891,4.247558,0.976100
2,(COCA-COLA LIGHT - Cálido),(COCA-COLA - Cálido),0.477155,0.546683,0.463647,0.971690,1.777431,0.202795,16.012772,0.836558
3,(COCA-COLA - Cálido),(COCA-COLA LIGHT - Cálido),0.546683,0.477155,0.463647,0.848110,1.777431,0.202795,3.442269,0.964865


## Situación 3

In [437]:
basket2 = df.pivot_table(index=['pedido'], columns='marca_cat', aggfunc='size', fill_value=0)
basket2 = (basket2 > 0).astype(int)

# Aplicar el algoritmo Apriori
frequent_itemsets2 = fpgrowth(basket2, min_support=0.70, use_colnames=True)

g:\Supermarket-Sales-Tracker-Mx\.venv\Lib\site-packages\mlxtend\frequent_patterns\fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [438]:
frequent_itemsets2

,support,itemsets
0,0.921335,(COCA-COLA - poco)
1,0.785856,(COCA-COLA LIGHT - poco)
2,0.811681,(JOYA - poco)
3,0.763607,"(COCA-COLA - poco, COCA-COLA LIGHT - poco)"
4,0.794597,"(JOYA - poco, COCA-COLA - poco)"


In [439]:
rules2 = association_rules(frequent_itemsets2, metric="confidence", min_threshold=0.5)

In [440]:
rules2[:]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(COCA-COLA - poco),(COCA-COLA LIGHT - poco),0.921335,0.785856,0.763607,0.828806,1.054653,0.039571,1.250880,0.658752
1,(COCA-COLA LIGHT - poco),(COCA-COLA - poco),0.785856,0.921335,0.763607,0.971689,1.054653,0.039571,2.778563,0.241991
2,(JOYA - poco),(COCA-COLA - poco),0.811681,0.921335,0.794597,0.978953,1.062537,0.046767,3.737506,0.312535
3,(COCA-COLA - poco),(JOYA - poco),0.921335,0.811681,0.794597,0.862441,1.062537,0.046767,1.369005,0.748189


## Evaluación de Modelos

Obtenemos resultados sino identicos muy similares, con ligeras variaciones que las primeras muestras solo dse detectan con variaciones en el orden, pero no con el valor de cada atributo respecto a su renglon, por lo tanto ambos algoritmos nos dan resultados de confianza y no hay uno mejor que otro para este caso en particular.

## Evaluación de Resultados

Los resultados indican que los compradores tienen un consumo mayor en temporadas frías coomo son el invierno o temporadas calidas en primavera para manejar la temperatura, por ende hay que enfocarnos más en las temporadas calidas que se combate el calor con los liquidos, así como en el invierno por la cantidad de fisestas presentes que hacen que aumente el consumo.